# PYNQ Project Proposal

##### __Proposer:__ Patrick Lysaght (patrick**dot**lysaght**at**xilinx**dot**com)  
__Version:__ 1.0  
__Date:__ 5 Oct 2017  

## Title

Automatically annotate state labels to waveforms captured from executed FSMs in `logictools`

### Abstract

Add support for displaying state labels to the waveforms generated by the FSM Generator in `logictools`

### Motivation

The waveforms captured with the `logictools` Trace Analyzer can show the sequential operation a FSM. We can capture and display the behavior of an FSM, as it sequences through a given set of its possible states, in response to any given set of input conditions.

Currently, however, the designer, must _manually_ decode each state from its corresponding state representation.  This entails checking the signals corresponding to the state bits, and repeatedly matching the values of signals to the appropriate states.  This can be a frustrating and error prone exercise. 

This process can be automated, making it much easier for the designer to interpret a waveform diagram.  Each FSM waveform will consist of a particular sequence of states.  These can be captured with the `logictools` Trace Analyzer, which can monitor an FSM as it executes.

Automatically annotating state labels to a waveform captured from a running FSM, provides both a useful pedagogical tool, as well as, a helpful debug tool.  

Having the FSM states automatically decoded and annotated on to the waveform diagram, helps when debugging by making it easier to:
* visualize the operation of a FSM
* confirm that the intended FSM has been properly specified, or
* locate errors in the FSM operation caused either by improper specification, or unexpected operating conditions

### Skill levels required for project

__Python:__ intermediate
    
__Digital logic:__ intermediate

__WaveDrom__: intermediate

### Example

_`logictools`_ can capture the operation of a FSM.  It can then display the operation as a waveform.   This is achieved using  the Trace Analyzer to monitor the operation of the FSM Generator, _under the hood_.  The Wavedrom library is also invoked, _under the hood_, to display the waveform.  The following script provides an example of a 3-bit, up/down Gray code counter:

In [1]:
from pynq.overlays.logictools import LogicToolsOverlay


logictools_olay = LogicToolsOverlay('logictools.bit')

# Specify FSM to generate
gray_cntr_spec = {'inputs': [('reset','D0'), ('direction','D1')],
                  'outputs': [('bit2','D3'), ('bit1','D4'), ('bit0','D5')],
                  'states': ['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7'],
                  'transitions': [['01', 'S0', 'S1', '000'],
                                  ['00', 'S0', 'S7', '000'],
                                  ['01', 'S1', 'S2', '001'],
                                  ['00', 'S1', 'S0', '001'],
                                  ['01', 'S2', 'S3', '011'],
                                  ['00', 'S2', 'S1', '011'],
                                  ['01', 'S3', 'S4', '010'],
                                  ['00', 'S3', 'S2', '010'],
                                  ['01', 'S4', 'S5', '110'],
                                  ['00', 'S4', 'S3', '110'],
                                  ['01', 'S5', 'S6', '100'],
                                  ['00', 'S5', 'S4', '100'],
                                  ['01', 'S6', 'S7', '101'],
                                  ['00', 'S6', 'S5', '101'],
                                  ['01', 'S7', 'S0', '111'],
                                  ['00', 'S7', 'S6', '111'],                            
                                  ['1-', '*',  'S0', '']]}   
        
# Instantiate fsm_gen
fsm_gen = logictools_olay.fsm_generator
# Set trace to 16 samples initially
fsm_gen.trace(num_analyzer_samples=16)
# Configure fsm_gen with _spec
fsm_gen.setup(gray_cntr_spec)
# Run FSM, capture operation, and display waveform 
fsm_gen.run()
fsm_gen.show_waveform()


By default, the trace waveform show the FSM inputs and the FSM outputs.  To see the state vector bits, we need add an extra parameter in the `setup method` as follows:

In [2]:
# Reset fsm_gen before invoking setup again
fsm_gen.reset()

# Configure fsm_gen with _spec
fsm_gen.setup(gray_cntr_spec, use_state_bits=True)
# Run FSM, capture operation, and display waveform 
fsm_gen.run()
fsm_gen.show_waveform()


We can access the data dictionary that was used to generate this waveform with the following script:

In [3]:
from pprint import pprint

fsm_gen_wave_dict = fsm_gen.waveform.waveform_dict
pprint(fsm_gen_wave_dict)

{'foot': {'tock': 1},
 'head': {'text': 'Finite State Machine'},
 'signal': [['analysis',
             {'name': 'reset', 'pin': 'D0', 'wave': 'l...............'},
             {'name': 'direction', 'pin': 'D1', 'wave': 'h...............'},
             {'name': 'bit2', 'pin': 'D3', 'wave': 'l...h...l...h...'},
             {'name': 'bit1', 'pin': 'D4', 'wave': 'l.h..l.hl.h..l.h'},
             {'name': 'bit0', 'pin': 'D5', 'wave': 'lh.l..h.lh.l..h.'},
             {'name': 'state_bit2', 'pin': 'D7', 'wave': 'l...h...l...h...'},
             {'name': 'state_bit1', 'pin': 'D6', 'wave': 'l.h.l.h.l.h.l.h.'},
             {'name': 'state_bit0', 'pin': 'D2', 'wave': 'lhlhlhlhlhlhlhlh'}]]}


This format of the data returned is one which is specified by the `WaveDrom` library.  This is because, internally, `Waveform` objects are represented using `WaveDrom` format, and rendered using `WaveDrom` methods.

The format is referred to as WaveDrom's WaveJSON format. For more details on how this works, refer to the [WaveDrom waveform tutorial](http://wavedrom.com/tutorial.html)

Since this data is simply a WaveJSON dict, we can extend it with any additional annotations that we would find helpful.  In this case, we want to add the label for each state the FSM sequences through.

In [4]:
# Let's annotate the first 16 states manually as an example
expected_state_seq = ['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7',
                      'S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7']
no_of_states = len(expected_state_seq)

fsm_gen_wave_dict['signal'].extend([{},
                                ['FSM states',
                                 {},
                                 {'name': 'state_label',
                                  'wave': '2' * no_of_states,
                                  'data': expected_state_seq},
                                 {}
                                 ]])

# Inspect the new WaveDrom structure
pprint(fsm_gen_wave_dict)

{'foot': {'tock': 1},
 'head': {'text': 'Finite State Machine'},
 'signal': [['analysis',
             {'name': 'reset', 'pin': 'D0', 'wave': 'l...............'},
             {'name': 'direction', 'pin': 'D1', 'wave': 'h...............'},
             {'name': 'bit2', 'pin': 'D3', 'wave': 'l...h...l...h...'},
             {'name': 'bit1', 'pin': 'D4', 'wave': 'l.h..l.hl.h..l.h'},
             {'name': 'bit0', 'pin': 'D5', 'wave': 'lh.l..h.lh.l..h.'},
             {'name': 'state_bit2', 'pin': 'D7', 'wave': 'l...h...l...h...'},
             {'name': 'state_bit1', 'pin': 'D6', 'wave': 'l.h.l.h.l.h.l.h.'},
             {'name': 'state_bit0', 'pin': 'D2', 'wave': 'lhlhlhlhlhlhlhlh'}],
            {},
            ['FSM states',
             {},
             {'data': ['S0',
                       'S1',
                       'S2',
                       'S3',
                       'S4',
                       'S5',
                       'S6',
                       'S7',
                 

We have created a dummy signal group called `FSM States` which we have used to annotate the waveform with the labels for each state.  In this simple case, we knew the expected states in advance, so we exploited that information, by creating the list `expected_state_seq`.

The key:value pair syntax (`data`, `name`, `wave`) used in the FSM states is specified by `WaveDrom's` `WaveJSON` format.  Note that this is more commonly used to specify buses. See the [WaveDrom waveform tutorial](http://wavedrom.com/tutorial.html) for more details.  In this example, we have re-purposed it, to annotate the FSM waveform with state labels.

The empty pairs of braces, '{}', create additional blank lines to separate the signal traces from the state labels.

Now let's see the result of extending the captured waveform with the FSM state label annotations:

In [5]:
from pynq.lib.logictools import Waveform

annotated_fsm_gen_waveform = Waveform(fsm_gen_wave_dict)
annotated_fsm_gen_waveform.display()

#### Extending the example

### Next steps

* __Write a script that will automatically re-generate the state encoding table for any FSM__

An object of type `FSM_Generator` has a number of attributes that can help us to re-create its state encoding table.  These are 

* `state_names`
* `num_states`
* `num_state_bits`

The script below shows the values of these attributes for `fsm_gen`, the  FSM generator object, which we declared in our earlier example.

In [6]:
print(' fsm_gen.state_names:\t\t', fsm_gen.state_names, '\n',
      'fsm_gen.num_states:\t\t', fsm_gen.num_states, '\n',
      'fsm_gen.num_state_bits:\t', fsm_gen.num_state_bits)

 fsm_gen.state_names:		 ['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7'] 
 fsm_gen.num_states:		 8 
 fsm_gen.num_state_bits:	 3


Note that objects of the the `FSM_Generator` class are assigned state codes sequentially, starting from binary 0, according to the order in which the states are specified.  So for `fsm_gen`, whose states were specified earlier, in the order shown below: 

```python
# Specify FSM to generate
fsm_spec = {'inputs': [('reset','D0'), ('direction','D1')],
            'outputs': [('bit2','D3'), ('bit1','D4'), ('bit0','D5')],
            'states': ['S0', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7'],
```

The state encoding table is as follows:

State encoding | State label
---------------|---------------
000            |S0  
001            |S1  
010            |S2  
011            |S3  
100            |S4  
101            |S5  
110            |S6  
111            |S7  


* __Use the state encoding table, to parse captured FSM waveform dictionary to identify the state sequences__

We can parse the values of the state bit signals in the waveform dictionary, for each clock cycle.  In this way, we can build up a sequence of state bit vectors; one vector for every state that the FSM has entered during the current execution trace.  Then, using the state encoding table, we can look-up the label for each state.

* __Post annotate the WaveJSON waveform trace with the state label information, in the style of the example, shown earlier__

The 3-bit Gray code counter example can be used as an initial test for the automatic annotation scripts.  The first goal would be to reproduce the FSM state labels, as shown above, for the Gray code counter.  Next, try reversing the count direction, by tying the `Direction` input pin low.  Verify that the state sequence is reversed, and that the new state labels match the reversed state sequence.

### Additional Resources

More details and examples of `WaveDrom`'s rendering capabilities for waveforms, and the WaveJSON format for specifying waveforms, are described in the `WaveDrom` [WaveDrom waveform tutorial](http://wavedrom.com/tutorial.html).

You can manually render waveforms with `WaveDrom's` online, interactive [WaveDrom editor](http://WaveDrom.com/editor.html).

----